## Building a BATTLE Eval

This notebook shows how to:
- Build and run an eval
- Load the results and into a Pandas Dataframe

In [39]:
initial_message = [{"role": "user", "content": ""}]

In [8]:
#!cd evals
#!git lfs fetch --all
#!git lfs pull

In [40]:
import os
from openai import OpenAI
import pandas as pd
import yaml
#from dotenv import load_dotenv

#load_dotenv()  # Load environment variables from .env file

#api_key = os.environ.get("OPENAI_API_KEY")
#client = OpenAI()

# Install Evals if you haven't already
# %pip install -e ../.
# pip install --upgrade openai
# %pip install pandas


In [42]:
# Paths. Assuming this notebook is in examples/

evals_path = os.path.join(os.getcwd(), "..", "evals")

registry_path = os.path.join(evals_path, "registry", "evals", "battles.yaml")

data_path = os.path.join(evals_path, "registry", "data", "battles")
os.makedirs(data_path, exist_ok=True)
data_path = os.path.join(data_path, "samples.jsonl")

json_logs_path = os.path.join(evals_path, "evallogs")
os.makedirs(json_logs_path, exist_ok=True)
json_logs_path = os.path.join(evals_path, "evallogs", "logs")

df_path = os.path.join(evals_path, "evallogs", "df")

In [43]:
# Registry yaml

registry_yaml = {}

registry_yaml["battles"] = {
    "id": "battles.test.v1",
    "metrics": ["accuracy"]
}
registry_yaml["battles.test.v1"] = {
    "class": "evals.elsuite.modelgraded.classify:ModelBasedClassify",
    "args": {
        "samples_jsonl": "battles/samples.jsonl",
        "eval_type": "cot_classify",
        "modelgraded_spec": "battle"
    }
}
with open(os.path.join(registry_path), "w") as f:
    yaml.dump(registry_yaml, f)

In [44]:
# Data structure

dataset = []
candidate_messages = [initial_message]
next_candidate_messages = []
best_response = []
best_message = [initial_message]

In [52]:
def battle(best_message, candidate_message):

    dataset = [{"input1": candidate_message, "input2": best_message}]

    df = pd.DataFrame(dataset)
    df.to_json(data_path, orient="records", lines=True)

    !oaieval gpt-3.5-turbo test-match
    
    with open(json_logs_path, "r") as f:
        events_df = pd.read_json(f, lines=True)

    os.makedirs(record_path, exist_ok=True)
    events_df.to_json(os.path.join(record_path, "events"), lines=True, orient="records")
    events_df["data"].to_json(os.path.join(df_path, "data"), lines=True, orient="records")
    data_df = pd.read_json(os.path.join(df_path, "data"), lines=True)
    for row in data_df[0]:
        s = str(row)
        s = s.replace("[","",-1)
        s = s.replace("]","",-1)
        if s.startswith("{'choice':"):
            s = s.split("'")[3]
            choice = s
        if s.startswith("{'prompt': {'role': 'user', 'content': 'You are comparing two responses to the following two instructions."):
            s = s.split("\\n\\nInstruction 1\\n",)[1]
            instruction1 = s.split("\\n\\nResponse 1\\n")[0].replace("\\'","'").replace("\\n","\n")
            s = s.split("\\nResponse 1\\n",)[1]
            response1 = s.split("\\n\\nInstruction 2\\n")[0].replace("\\'","'").replace("\\n","\n")
            s = s.split("\\n\\nInstruction 2\\n",)[1]
            instruction2 = s.split("\\n\\nResponse 2\\n")[0].replace("\\'","'").replace("\\n","\n")
            s = s.split("\\nResponse 2\\n",)[1]
            response2 = s.split("\\n\\n\\nIs the first response better than the second?")[0].replace("\\'","'").replace("\\n","\n")
            sampled = s.split("\\n\\nReasoning:\'}, \'sampled\': ")[1].replace("\\'","'").replace("\\n","\n")
    data = {'Instruction1': instruction1, 'Response1': response1, 'Instruction2': instruction2, 'Response2': response2, 'Sampled': sampled, 'Choice': choice}
    return data

In [46]:
def switch_system_and_user(message):
    new_message = []
    if message[0]["role"] == "system":
        new_message.append({"role": "user", "content": message[0]["content"]})
    else:
        new_message.append({"role": "system", "content": message[0]["content"]})
    return new_message

In [53]:
for candidate_message in candidate_messages:
    data = battle(best_message, candidate_message)
    if data["Choice"] == "Yes":
        best_message.append(candidate_message)
        best_response.append(data["Response1"]) 
    dataset.append(data)
    new_message = {"role": candidate_message["role"], "content":data["Response1"]}
    if new_message not in next_candidate_messages:
        next_candidate_messages.append(new_message)
        next_candidate_messages.append(switch_system_and_user(new_message))

[2024-08-05 19:03:25,781] [registry.py:271] Loading registry from /Users/janvotava/Desktop/evals/evals/registry/evals
[2024-08-05 19:03:26,808] [registry.py:271] Loading registry from /Users/janvotava/.evals/evals
[2024-08-05 19:03:26,811] [oaieval.py:215] Run started: 240805170326MT3T4N4C
[2024-08-05 19:03:27,014] [eval.py:36] Evaluating 3 samples
[2024-08-05 19:03:27,033] [eval.py:144] Running in threaded mode with 10 threads!
100%|█████████████████████████████████████████████| 3/3 [00:01<00:00,  2.01it/s]
[2024-08-05 19:03:28,545] [oaieval.py:275] Found 3/3 sampling events with usage data
[2024-08-05 19:03:28,546] [oaieval.py:283] Token usage from 3 sampling events:
completion_tokens: 14
prompt_tokens: 80
total_tokens: 94
[2024-08-05 19:03:28,547] [record.py:371] Final report: {'accuracy': 0.6666666666666666, 'boostrap_std': 0.4775939698111775, 'usage_completion_tokens': 14, 'usage_prompt_tokens': 80, 'usage_total_tokens': 94}. Logged to /tmp/evallogs/240805170326MT3T4N4C_gpt-3.5-tu

FileNotFoundError: [Errno 2] No such file or directory: '/Users/janvotava/Desktop/evals/examples/../evals/evallogs/logs'